In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.base import Int, Str, Float
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation

from aiida.orm.data.parameter import ParameterData

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

# Select structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    update_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))

def update_view():

    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    
    structure = struct_browser.results.value
    
    if structure:
        atoms = structure.get_ase()

        # add new component
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center_view()

# Select computer and codes

In [ ]:
from aiida.orm.code import Code
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.work.run import submit
from aiida.orm.data.structure import StructureData


qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name')
computer_names = [comp[0] for comp in qb.all()]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': plugin_name})
    qb.order_by({Code: {'id': 'asc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    global cp2k_codes, eval_morb_codes, stm_image_codes
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    eval_morb_codes = comp_plugin_codes(drop_computer.value, 'stm.eval_morbs')
    stm_image_codes = comp_plugin_codes(drop_computer.value, 'stm.stm_image')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_eval.options = [c.label for c in eval_morb_codes]
    drop_imag.options = [c.label for c in stm_image_codes]
    
    
drop_computer.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_eval = ipw.Dropdown(description="Eval code")

drop_imag = ipw.Dropdown(description="Image code")

on_computer_change(0)

display(drop_computer, drop_cp2k, drop_eval, drop_imag)

# STM parameters

Parameters for CH and CC STM pictures

In [ ]:
style = {'description_width': '140px'}
layout = {'width': '50%'}
layout_small = {'width': '25%'}

bias_voltages_text = ipw.Text(description='Voltages (V)',
                              value='-1.0 -0.5 -0.1 0.1 0.5 1.0',
                              style=style, layout=layout)

const_height_text = ipw.Text(description='Const. H (ang)',
                              value='3.0 5.0',
                              style=style, layout=layout)

const_current_text = ipw.Text(description='Const. cur. (isoval)',
                              value='1e-8 1e-6',
                              style=style, layout=layout)

extrap_plane_text = ipw.BoundedFloatText(
                        description='Extrap plane (ang)',
                        min=1.0,
                        max=10.0,
                        step=0.1,
                        value=3.0,
                        style=style, layout=layout_small)


display(bias_voltages_text, const_height_text, const_current_text, extrap_plane_text)

Parameters for CH pictures of individual orbitals

In [ ]:
num_homo_text = ipw.IntText(
                    value=5,
                    description="num HOMO",
                    style=style, layout=layout_small)
num_lumo_text = ipw.IntText(
                    value=5,
                    description="num LUMO",
                    style=style, layout=layout_small)

display(num_homo_text, num_lumo_text)

Parameters for STS

In [ ]:
sts_heights_text = ipw.Text(
                        description='STS heights (ang)',
                        value="3.0 5.0",
                        style=style, layout=layout)

sts_de_text = ipw.BoundedFloatText(
                        description='STS dE (eV)',
                        min=0.01,
                        max=0.10,
                        step=0.01,
                        value=0.05,
                        style=style, layout=layout_small)

sts_fwhm_text = ipw.BoundedFloatText(
                        description='STS FWHM (eV)',
                        min=0.05,
                        max=0.50,
                        step=0.01,
                        value=0.10,
                        style=style, layout=layout_small)

display(sts_heights_text, sts_de_text, sts_fwhm_text)

In [ ]:
from apps.stm.stm_workchain import STMWorkChain

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        
        bias_voltages = np.array(bias_voltages_text.value.split(), dtype=np.float)
        height_values = np.array(const_height_text.value.split(), dtype=np.float)
        current_values = np.array(const_current_text.value.split(), dtype=np.float)

        extrap_plane = float(extrap_plane_text.value)
        num_homo = int(num_homo_text.value)
        num_lumo = int(num_lumo_text.value)

        sts_heights = np.array(sts_heights_text.value.split(), dtype=np.float)
        sts_de = float(sts_de_text.value)
        sts_fwhm = float(sts_fwhm_text.value)
        
        parent_dir = "parent_calc/"

        eval_orbs_params = ParameterData(dict={
            '--cp2k_input':  parent_dir+'aiida.inp',
            '--basis_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':    parent_dir+'geom.xyz',
            '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
            '--output_file': 'morbs_dx0.2',
            '--emin':        np.min(bias_voltages)-0.1,
            '--emax':        np.max(bias_voltages)+0.1,
            '--eval_region': ['G', 'G', 'G', 'G', 'b-2.0_C', 't%.1f'%extrap_plane],
            '--dx':          '0.2',
            '--eval_cutoff': '14.0'
        })

        stm_image_params = ParameterData(dict={
            '--npz_file':          parent_dir+'morbs_dx0.2.npz',
            '--hartree_file':      parent_dir+parent_dir+'aiida-HART-v_hartree-1_0.cube',
            '--extrap_plane':      extrap_plane,
            '--extrap_extent':     '8.0',
            '--output_dir':        'stm_output',
        #
            '--bias_voltages':     bias_voltages,
            '--stm_plane_heights': height_values,
            '--stm_isovalues':     current_values,
        #
            '--sts_plane_heights': sts_heights,
            '--sts_de':            sts_de,
            '--sts_fwhm':          sts_fwhm,
            '--sts_elim':          [np.min(bias_voltages), np.max(bias_voltages)],
        #
            '--orb_plane_heights': height_values,
            '--n_homo':            num_homo,
            '--n_lumo':            num_lumo
        })
        
        cp2k_code = cp2k_codes[drop_cp2k.index]
        eval_orbs_code = eval_morb_codes[drop_eval.index]
        stm_image_code = stm_image_codes[drop_imag.index]
        
        struct = struct_browser.results.value
        
        outputs = submit(STMWorkChain,
                 cp2k_code=cp2k_code,
                 structure=struct,
                 eval_orbs_code=eval_orbs_code,
                 eval_orbs_params=eval_orbs_params,
                 stm_image_code=stm_image_code,
                 stm_image_params=stm_image_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)